# Computational Theory Assessment
---

This notebook explores the key components of the SHA-256 cryptographic hash algorithm as defined in the [Secure Hash Standard (FIPS 180-4)](https://nvlpubs.nist.gov/nistpubs/fips/nist.fips.180-4.pdf). It systematically builds SHA-256 from its fundamental bitwise operations through the complete compression function.

All implementations follow FIPS 180-4 specifications and include comprehensive testing with verification against official standards. **Rather than treating SHA-256 as a black box, this sequential approach reveals how simple mathematical operations combine to create complex cryptographic security properties.**

## Overview of Problems
I begin by examining the seven core bitwise functions defined in the **Secure Hash Standard**. These functions introduce **non-linear mixing** in the compression function, which ensures that even a small change in the input produces a large, **unpredictable** change in the output, technically known as **the avalanche effect**. It is important to note that although we cover the Parity function, **it** is only used in SHA-1, which is currently being phased out due to **successful collision attacks**.

Next, I **extract** the 64 round constants from the cube roots of prime numbers. This demonstrates the **"nothing up my sleeve"** principle, meaning these constants come from a transparent, publicly verifiable source rather than being secretly chosen. **This design choice ensures no hidden backdoors exist in the algorithm.**

Once the constants are established, I implement **message padding**. Padding ensures messages fit the required 512-bit block size by appending a single '1' bit, zero padding, and a 64-bit length field **encoding the original message length**. This structured padding guarantees that the message is unambiguous and can be processed in fixed size blocks.

With the message padded, I process it **block by block**. Each 512-bit block is expanded into a 64-entry message schedule, and eight working variables (a–h) are initialized from the current hash state. Over 64 rounds, these variables are updated using the message schedule, round constants, and the core **functions** (Ch, Maj, Sigma0, Sigma1).

Finally, I explore password security through practical demonstration. Using SHA-256, I successfully recover three common passwords via dictionary attack, revealing why fast hash functions are inappropriate for password storage. This section demonstrates the critical importance of purpose built password hashing functions like Argon2id, scrypt, bcrypt that incorporate salting and intentional computational cost.

## Table of Contents
- Problem 1: Binary Words and Operations
- Problem 2: Fractional Parts of Cube Roots
- Problem 3: Padding
- Problem 4: Hashes
- Problem 5: Passwords

## Python Imports Used

In [1]:
# Import necessary libraries

# NumPy is used throughout this notebook for:
#   • handling 32-bit unsigned integers (np.uint32) to match SHA-256 word operations,
#   • mathematical functions such as np.floor and np.cbrt for computing constants,
#   • controlling numerical behaviour via np.seterr.
# Official NumPy documentation: https://numpy.org/doc/2.3/user/index.html#user
import numpy as np


# hashlib is part of Python's standard library.
# It is used in this notebook for two purposes:
#   • Problem 4: to verify the correctness of the custom SHA-256 implementation
#     by comparing its output to hashlib.sha256().
#   • Problem 5: to compute SHA-256 hashes of candidate passwords during the
#     dictionary-attack demonstration.
# Documentation: https://docs.python.org/3/library/hashlib.html
import hashlib 

## Problem 1: Binary Words and Operations

### Implement the following functions in Python. Use **NumPy** to ensure that all variables and values are treated as **32-bit integers**. These functions are defined in the **Secure Hash Standard (see page 10)**.

1. `Parity(x, y, z)`
2. `Ch(x, y, z)`
3. `Maj(x, y, z)`
4. `Sigma0(x)` - *written as $\Sigma_0^{256}(x)$ in the standard.*
5. `Sigma1(x)` - *written as $\Sigma_1^{256}(x)$ in the standard.*
6. `sigma0(x)` - *written as $\sigma_0^{256}(x)$ in the standard.*
7. `sigma1(x)` - *written as $\sigma_1^{256}(x)$ in the standard.*

### Instructions

- Document each function with a clear docstring
- Explain its purpose and behaviour in Markdown
- Test it with appropriate examples to verify correctness.

### Solution:

### 1. Parity(x, y, z) Function (Used in SHA-1 - deprecated)

The Parity function is a linear bitwise operation used in certain rounds of the SHA-1 hash algorithm, specifically rounds 20-39 and 60-79 as stated in the Secure Hash Standard [(see page 10)](https://nvlpubs.nist.gov/nistpubs/fips/nist.fips.180-4.pdf). It takes three 32-bit words `x`, `y`, and `z`, and combines them by applying the XOR operation to each corresponding bit. 

$$
Parity(x, y, z) = x \oplus y \oplus z
$$

XOR (exclusive OR) works like this: for each bit position, if the number of `1`s in that position across `x`, `y`, and `z` is **odd**, the result bit is `1`. If it is **even**, including the case where there are only 0's, then the result bit is `0`. [See here for more information on XOR](https://www.geeksforgeeks.org/software-engineering/bitwise-xor-operator-in-programming/).

Table displaying how XOR works:
| x | y | z | Number of 1's Odd? | Parity |
| - | - | - | ------------------ | ------ |
| 0 | 0 | 0 | No                 | 0      |
| 0 | 0 | 1 | Yes                | 1      |
| 0 | 1 | 1 | No                 | 0      |
| 1 | 1 | 1 | Yes                | 1      |

However, despite these design features, **SHA-1 is no longer considered secure due to successful collision attacks**. A significant milestone was a `2015` practical attack on SHA‑1 referred to as `The SHAppening`, which showed that attacks requiring around `2^57` operations were feasible in practice. This and related work undermined confidence in SHA‑1 and paved the way for the `2017 SHAttered attack`, the first publicly demonstrated practical collision for the full SHA‑1 hash function. [This document released by git](https://git-scm.com/docs/BreakingChanges/2.46.0) contains information on changes being made to move from SHA-1 to SHA-256, and lists the attacks made on SHA-1. 

Today, major organizations have phased out SHA-1 in favor of stronger algorithms like SHA-2 and SHA-3, which do not use the Parity function. [See official news release from NIST](https://www.nist.gov/news-events/news/2022/12/nist-retires-sha-1-cryptographic-algorithm) stating that they will stop using SHA-1 by December 31st 2030 and anyone relying on SHA-1 should migrate to SHA-2 or SHA-3 as soon as possible. 

In [2]:
def parity(x: int, y: int, z: int) -> np.uint32:
    """
    Performs a bitwise XOR (parity function) across three 32-bit integers.

    This function is used in the SHA-1 algorithm during rounds 20-39 and 60-79.

    Parameters:
        x (int): The first 32-bit integer.
        y (int): The second 32-bit integer.
        z (int): The third 32-bit integer.

    Returns:
        np.uint32: The result of x XOR y XOR z, as a 32-bit unsigned integer.
    """

    # Ensure inputs are treated as 32-bit unsigned integers
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)

    return x ^ y ^ z


In [3]:
# Test cases to validate the Parity function
def test_parity():
    test_cases = [
        # Basic tests (decimal)
        ((0, 0, 0), 0), 
        ((1, 0, 0), 1),
        ((1, 1, 0), 0),
        ((1, 1, 1), 1),

        # Bit pattern tests (binary)
        ((0b1010, 0b0101, 0b1111), 0b0000),
        ((0b1100, 0b0011, 0b0101), 0b1010),
        ((0b1111, 0b1111, 0b1111), 0b1111),

        # Full 32-bit tests (hex)
        ((0xFFFFFFFF, 0x00000000, 0x00000000), 0xFFFFFFFF),
        ((0xFFFFFFFF, 0xFFFFFFFF, 0x00000000), 0x00000000),
        ((0xFFFFFFFF, 0xFFFFFFFF, 0xFFFFFFFF), 0xFFFFFFFF),
        ((0xAAAAAAAA, 0x55555555, 0xFFFFFFFF), 0x00000000),

        # Edge cases (hex)
        ((0x00000000, 0x00000000, 0x00000000), 0x00000000),
    ]

    for (x, y, z), expected in test_cases:
        result = parity(x, y, z)
        assert result == expected, (
            f"Expected {hex(expected)}, got {hex(result)} "
            f"for parity({hex(x)}, {hex(y)}, {hex(z)})"
        )

    # Final complex value test
    x, y, z = 0x12345678, 0x87654321, 0xDEADBEEF
    expected = np.uint32(x) ^ np.uint32(y) ^ np.uint32(z)
    result = parity(x, y, z)
    assert result == expected, (
        f"Expected {hex(expected)}, got {hex(result)} "
        f"for parity({hex(x)}, {hex(y)}, {hex(z)})"
    )

    print("All parity function test cases passed.")

test_parity()

All parity function test cases passed.


### 2. Ch(x, y, z) Function (Used in SHA-1 and all SHA-2 variants)

The Ch (Choice) function is a nonlinear bitwise operation used in both SHA-1 (specifically in rounds 0-19) and all SHA-2 variants (in all rounds 0 to 63). It takes three 32-bit (or 64-bit, depending on the SHA-2 variant) words as input: `x`, `y`, and `z`. [See section 4.1.1](https://nvlpubs.nist.gov/nistpubs/fips/nist.fips.180-4.pdf).

$$
Ch(x, y, z) = (x \land y) \oplus (\lnot x \land z)
$$

The Ch function operates as a bitwise selector or multiplexer:
- If the bit in `x` is `1`, the function selects the corresponding bit from `y`
- If the bit in `x` is `0`, the function selects the corresponding bit from `z`

Table displaying how the Ch function works: 
| x | y | z | Ch(x, y, z) | Explanation                      |  
| - | - | - | ----------- | -------------------------------- |  
| 0 | 0 | 0 | 0           | x=0 → select z=0                 |  
| 0 | 0 | 1 | 1           | x=0 → select z=1                 |  
| 0 | 1 | 1 | 1           | x=0 → select z=1                 |  
| 1 | 0 | 0 | 0           | x=1 → select y=0                 |  
| 1 | 1 | 0 | 1           | x=1 → select y=1                 |  
| 1 | 1 | 1 | 1           | x=1 → select y=1                 |

This function is used in the round functions to mix and modify intermediate hash values (a, b, c, d, e, f, g, h) during each round of processing, specifically affecting `e`, `f`, and `g`. The value of `e` determines whether `f` or `g` gets selected in the update process. By incorporating this function, SHA-2 ensures that small changes in the input message result in unpredictable changes in the final hash value, contributing to the algorithm's security. [See section 6.2.2 step 3.](https://nvlpubs.nist.gov/nistpubs/fips/nist.fips.180-4.pdf)

The name "Ch" reflects its behavior as a "chooser" or "selector" between the bits of `y` and `z` based on `x`.

In [4]:
def ch(x: int, y: int, z: int) -> np.uint32:
    """
    Performs the Choice function on three 32-bit integers.
    
    This function is used in:
    - SHA-1 algorithm during rounds 0-19 
    - All SHA-2 variants
    
    Parameters:
        x (int): Selector 32-bit integer.
        y (int): Bits to be selected when x bit is 1.
        z (int): Bits to be selected when x bit is 0.

    Returns:
        np.uint32: The result of (x & y) ^ (~x & z), as a 32-bit unsigned integer.
    """
    # Ensure inputs are treated as 32-bit unsigned integers
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)

    return (x & y) ^ (~x & z)

In [5]:
# Test cases to validate the ch function
def test_ch():
    test_cases = [
        # Basic tests (decimal)
        ((0, 0, 0), 0), 
        ((1, 0, 0), 0),   
        ((1, 1, 0), 1),   
        ((1, 1, 1), 1),   

        # Bit pattern tests (binary)
        ((0b1010, 0b0101, 0b1111), 0b0101),  
        ((0b1100, 0b0011, 0b0101), 0b0001),  
        ((0b1111, 0b1111, 0b1111), 0b1111),  

        # Full 32-bit tests (hex)
        ((0xFFFFFFFF, 0xAAAAAAAA, 0x55555555), 0xAAAAAAAA),
        ((0x00000000, 0xAAAAAAAA, 0x55555555), 0x55555555),
        ((0xAAAAAAAA, 0xFFFFFFFF, 0x00000000), 0xAAAAAAAA),

        # Edge cases (hex)
        ((0x00000000, 0x00000000, 0x00000000), 0x00000000),
    ]
    
    for (x, y, z), expected in test_cases:
        result = ch(x, y, z)
        assert result == expected, (
            f"Expected {hex(expected)}, got {hex(result)} "
            f"for ch({hex(x)}, {hex(y)}, {hex(z)})"
        )
    
    # Complex value test
    x, y, z = 0x12345678, 0x87654321, 0xDEADBEEF
    expected = (np.uint32(x) & np.uint32(y)) ^ (~np.uint32(x) & np.uint32(z))
    result = ch(x, y, z)
    assert result == expected, (
        f"Expected {hex(expected)}, got {hex(result)} "
        f"for ch({hex(x)}, {hex(y)}, {hex(z)})"
    )
    
    print("All ch function test cases passed.")

# Run the tests
test_ch()

All ch function test cases passed.


### 3. Maj(x, y, z) Function (Used in SHA-1 and all SHA-2 variants)

The Maj (Majority) function is a nonlinear bitwise operation used in both SHA-1 (specifically rounds 40-59) and all SHA-2 variants (in all rounds 0 to 63). It takes three 32-bit (or 64-bit, depending on the SHA-2 variant) words as input: `x`, `y`, and `z`. [See section 4.1.1](https://nvlpubs.nist.gov/nistpubs/fips/nist.fips.180-4.pdf).

$$
Maj(x, y, z) = (x \land y) \oplus (x \land z) \oplus (y \land z)
$$

The Maj function implements a bit-wise majority vote: for each bit position, the output bit is `1` if at least two of the three input bits are `1`, otherwise it is `0`.

Table displaying how the Maj function works: 
| x | y | z | Maj(x, y, z) | Explanation                      |  
| - | - | - | -----------  | -------------------------------- |  
| 0 | 0 | 0 | 0            | Majority of bits is 0            |  
| 0 | 0 | 1 | 0            | Majority of bits is 0            |  
| 0 | 1 | 1 | 1            | Majority of bits is 1            |  
| 1 | 1 | 1 | 1            | Majority of bits is 1            |  

This function provides important nonlinear mixing in the hash algorithm by spreading influence across bits, ensuring that small changes in the input message cause widespread changes in the hash value. By creating dependency between the input bits, Maj helps diffuse changes and is a key factor in the avalanche effect, which is essential for the cryptographic security of SHA algorithms. [See section 6.2.2 step 3.](https://nvlpubs.nist.gov/nistpubs/fips/nist.fips.180-4.pdf)

The name "Maj" reflects its behavior as a "majority" selector that chooses the most common bit value among the three inputs at each position.


In [6]:
def maj(x: int, y: int, z: int) -> np.uint32:
    """
    Performs the Majority function on three 32-bit integers.
    
    This function is used in:
    - SHA-1 algorithm during rounds 40-59
    - All SHA-2 variants
    
    Parameters:
        x (int): The first 32-bit integer.
        y (int): The second 32-bit integer.
        z (int): The third 32-bit integer.

    Returns:
        np.uint32: The result of (x & y) ^ (x & z) ^ (y & z), as a 32-bit unsigned integer.
    """
    # Ensure inputs are treated as 32-bit unsigned integers
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    return (x & y) ^ (x & z) ^ (y & z)  

In [7]:
# Test cases to validate the Maj function
def test_maj():
    test_cases = [
        # Basic tests
        ((0, 0, 0), 0),
        ((1, 0, 0), 0),
        ((1, 1, 0), 1),
        ((1, 1, 1), 1),

        # Bit pattern tests (binary)
        ((0b1010, 0b0101, 0b1111), 0b1111),
        ((0b1100, 0b0011, 0b0101), 0b0101),
        ((0b1111, 0b1111, 0b1111), 0b1111),

        # Full 32-bit tests (hex)
        ((0xFFFFFFFF, 0x00000000, 0x00000000), 0x00000000),
        ((0xFFFFFFFF, 0xFFFFFFFF, 0x00000000), 0xFFFFFFFF),
        ((0xFFFFFFFF, 0xFFFFFFFF, 0xFFFFFFFF), 0xFFFFFFFF),
        ((0xAAAAAAAA, 0x55555555, 0xFFFFFFFF), 0xFFFFFFFF),

        # Edge cases (hex)
        ((0x00000000, 0x00000000, 0x00000000), 0x00000000),
    ]
    
    for (x, y, z), expected in test_cases:
        result = maj(x, y, z)
        assert result == expected, (
            f"Expected {hex(expected)}, got {hex(result)} "
            f"for maj({hex(x)}, {hex(y)}, {hex(z)})"
        )
    
    # Complex value test
    x, y, z = 0x12345678, 0x87654321, 0xDEADBEEF
    expected = (np.uint32(x) & np.uint32(y)) ^ (np.uint32(x) & np.uint32(z)) ^ (np.uint32(y) & np.uint32(z))
    result = maj(x, y, z)
    assert result == expected, (
        f"Expected {hex(expected)}, got {hex(result)} "
        f"for maj({hex(x)}, {hex(y)}, {hex(z)})"
    )
    
    print("All maj function test cases passed.")

# Run the tests
test_maj()

All maj function test cases passed.


### 4. Sigma0(x) Function (Used in SHA-256 and SHA-224)

The `Sigma0(x)` function is one of the two "uppercase sigma" functions used in the SHA-256 and SHA-224 compression rounds. It operates on a 32-bit word `x` and mixes its bits through rotations and XOR. [See section 4.1.2](https://nvlpubs.nist.gov/nistpubs/fips/nist.fips.180-4.pdf) for the function definition as outlined below.

$$
\Sigma_0^{256}(x) = ROTR^2(x) \oplus ROTR^{13}(x) \oplus ROTR^{22}(x)
$$

Here, `ROTR^n(x)` represents a circular right rotation of the bits in `x` by `n` positions.

My understanding is that the combination of these three distinct rotations by `2`, `13` and `22` bits is essential because by performing these three different circular shifts and then using XOR to combine the results, the function basically ensures that any change to a single bit in the input `x` is propagated to multiple, random bit positions in the output. This would contribute to the [avalanche effect](https://www.geeksforgeeks.org/computer-networks/avalanche-effect-in-cryptography/) which is fundamental to the cryptographic strength and security of the SHA-2 family of algorithms as a small change in the input produces significant changes to the output. 


In [8]:
def big_sigma0(x: int) -> np.uint32:
    """
    Performs the Σ₀ function on a 32-bit word.

    This function is used in:
    - The SHA-256 compression function 
    - The SHA-224 compression function

    The operation involves three circular right rotations (ROTR) of the input 
    word by 2, 13, and 22 bits, followed by a bitwise XOR of the results.

    Parameters:
        x (int): A 32-bit word.

    Returns:
        np.uint32: The result of ROTR²(x) ⊕ ROTR¹³(x) ⊕ ROTR²²(x), as a 32-bit unsigned integer.
    """
    x = np.uint32(x)

    # Perform the circular right rotations 
    rotr2 = np.uint32((x >> 2) | (x << 30)) # 32 - 2 = 30
    rotr13 = np.uint32((x >> 13) | (x << 19)) # 32 - 13 = 19
    rotr22 = np.uint32((x >> 22) | (x << 10)) # 32 - 22 = 10

    return rotr2 ^ rotr13 ^ rotr22


In [9]:
# Test cases to validate the big_sigma0 function
def test_big_sigma0():
  
    test_cases = [
        (0x00000000, 0x00000000), 
        (0xFFFFFFFF, 0xffffffff), 
        (0x80000000, 0x20040200), 
        (0x12345678, 0x66146474), 
    ]

    for x, expected in test_cases:
        result = big_sigma0(x)
        assert result == expected, (
            f"Test failed for input {hex(x)}: "
            f"expected {hex(expected)}, got {hex(result)}"
        )

    print("All big_sigma0 test cases passed.")

# Run the test
test_big_sigma0()

All big_sigma0 test cases passed.


### 5. Sigma1(x) Function (Used in SHA-256 and SHA-224)

The `Sigma1(x)` function is the second of the two "uppercase sigma" functions used int the SHA-256 and SHA-224 compression rounds. It operates on a 32-bit word `x` and mixes its bits through rotations and XOR. [See section 4.1.2](https://nvlpubs.nist.gov/nistpubs/fips/nist.fips.180-4.pdf) for the function definition as outlined below.

$$
\Sigma_1^{256}(x) = ROTR^6(x) \oplus ROTR^{11}(x) \oplus ROTR^{25}(x)
$$

Here, `ROTR^n(x)` represents a circular right rotation of the bits in `x` by `n` positions.

My understanding is that the three rotations by `6`, `11`, and `25` bits are another set of cryptographically-optimized constants. The reason for using these three specific, different rotations combined via XOR is to create a second, powerful diffusion layer. Just like Sigma0(x), this ensures that a single bit difference in the input is scattered to three different locations in the 32-bit word. This complex mixing is again, vital for achieving the strong [avalanche effect](https://www.geeksforgeeks.org/computer-networks/avalanche-effect-in-cryptography/), contributing to the cryptographic strength and security of the SHA-2 family of algorithms. 

In [10]:
def big_sigma1(x: int) -> np.uint32:
    """
    Performs the Σ₁ function on a 32-bit word.

    This function is used in:
    - The SHA-256 compression function
    - The SHA-224 compression function

    The operation involves three circular right rotations (ROTR) of the input
    word by 6, 11, and 25 bits, followed by a bitwise XOR of the results.

    Parameters:
        x (int): A 32-bit word.

    Returns:
        np.uint32: The result of ROTR⁶(x) ⊕ ROTR¹¹(x) ⊕ ROTR²⁵(x), as a 32-bit unsigned integer.
    """
    x = np.uint32(x)

    # Perform the circular right rotations
    rotr6 = np.uint32((x >> 6) | (x << 26))  # 32 - 6 = 26
    rotr11 = np.uint32((x >> 11) | (x << 21))  # 32 - 11 = 21
    rotr25 = np.uint32((x >> 25) | (x << 7))  # 32 - 25 = 7

    return rotr6 ^ rotr11 ^ rotr25

In [11]:
# Test cases to validate the big_sigma1 function
def test_big_sigma1():
  
    test_cases = [
        (0x00000000, 0x00000000), 
        (0xFFFFFFFF, 0xffffffff), 
        (0x80000000, 0x2100040), 
        (0x12345678, 0x3561abda), 
    ]

    for x, expected in test_cases:
        result = big_sigma1(x)
        assert result == expected, (
            f"Test failed for input {hex(x)}: "
            f"expected {hex(expected)}, got {hex(result)}"
        )

    print("All big_sigma1 test cases passed.")

# Run the test
test_big_sigma1()

All big_sigma1 test cases passed.


**It is important to note** that the uppercase sigma functions rotate the bits to the right and any bits at the end just wrap around to the left so none are lost. 

Now we will look at the lowercase sigma functions that also rotate the bits to the right but also shift them to the right as well. This shift causes the bits to be lost as they basically fall off the end instead of wrapping around like the ROTR operation.

### 6. sigma0(x) Function (Used in SHA-256 and SHA-224)

The `sigma0(x)` function is one of the two "lowercase sigma" nonlinear bitwise operations used in the SHA-256 and SHA-224 hash algorithms. It operates on a single 32-bit word and is used in the message schedule which can be seen in section [6.2.2 Part 1](https://nvlpubs.nist.gov/nistpubs/fips/nist.fips.180-4.pdf) of the official standard. [See section 4.1.2](https://nvlpubs.nist.gov/nistpubs/fips/nist.fips.180-4.pdf) for the function definition as outlined below.

$$
\sigma_0^{256}(x) = ROTR^7(x) \oplus ROTR^{18}(x) \oplus SHR^3(x)
$$

Here `ROTR^n(x)` represents a circular right rotation of the bits in `x` by `n` positions, and `SHR^n(x)` represents a right shift of `x` by `n` positions (filling with zeros on the left).

My understanding is that the function combines two circular rotations of `7` and `18` with one logical right shift of `3` via XOR and that the inclusion of the **shift right** operation is crucial because it irreversibly discards the least significant 3 bits of the word, which breaks the linear dependence between words. This deliberate information loss prevents simple mathematical analysis and dramatically enhances the **avalanche effect** by ensuring that differences in the initial message words are mixed in a complex, non-reversible way before they enter the main 64-round compression process.


In [12]:
def small_sigma0(x: int) -> np.uint32:
    """
    Performs the σ₀ function on a 32-bit word.

    This function is used in the SHA-256 and SHA-224 message schedule expansion.

    The function performs two circular right rotations (ROTR) of the input
    by 7 and 18 positions, one right shift (SHR) by 3 positions, followed 
    by a bitwise XOR of the results.
    
    Parameters:
        x (int): A 32-bit word
        
    Returns:
        np.uint32: The result of ROTR⁷(x) ⊕ ROTR¹⁸(x) ⊕ SHR³(x), as a 32-bit unsigned integer
    """
    # Ensure input is treated as a 32-bit unsigned integer
    x = np.uint32(x)
    
    # Perform circular right rotations by 7 and 18 bits
    rotr7 = np.uint32((x >> 7) | (x << 25)) # 32 - 7 = 25
    rotr18 = np.uint32((x >> 18) | (x << 14)) # 32 - 18 = 14

    # Perform right shift by 3 bits
    # SHR^n(x) = x >> n (fills with zeros on the left)
    shr3 = np.uint32(x >> 3)
    
    # XOR all three values together
    return rotr7 ^ rotr18 ^ shr3


In [13]:
# Test cases to validate the small_sigma0 function
def test_small_sigma0():
  
    test_cases = [
        (0x00000000, 0x00000000), 
        (0xFFFFFFFF, 0x1fffffff), 
        (0x80000000, 0x11002000), 
        (0x12345678, 0xe7fce6ee), 
    ]

    for x, expected in test_cases:
        result = small_sigma0(x)
        assert result == expected, (
            f"Test failed for input {hex(x)}: "
            f"expected {hex(expected)}, got {hex(result)}"
        )

    print("All small_sigma0 test cases passed.")

# Run the test
test_small_sigma0()

All small_sigma0 test cases passed.


### 7. sigma1(x) Function (Used in SHA-256 and SHA-224

The `sigma1(x)` function is the second of the two "lowercase sigma" nonlinear bitwise operations used in the SHA-256 and SHA-224 hash algorithms. It operates on a single 32-bit word and is used in the message schedule which can be seen in section [6.2.2 Part 1](https://nvlpubs.nist.gov/nistpubs/fips/nist.fips.180-4.pdf) of the official standard. [See section 4.1.2](https://nvlpubs.nist.gov/nistpubs/fips/nist.fips.180-4.pdf) for the function definition as outlined below.

$$
\sigma_1^{256}(x) = ROTR^{17}(x) \oplus ROTR^{19}(x) \oplus SHR^{10}(x)
$$

Here `ROTR^n(x)` represents a circular right rotation of the bits in `x` by `n` positions, and `SHR^n(x)` represents a right shift of `x` by `n` positions (filling with zeros on the left).

My understanding is that the function combines two circular rotations of `17` and `19` with one logical right shift of `10` via XOR. Similar to sigma0(x), the inclusion of the **shift right** operation is the key feature, as it irreversibly discards the least significant 10 bits of the word. This deliberate, non-reversible information loss breaks the simple linear dependence between message words. This design dramatically enhances the **avalanche effect** by ensuring that differences in the message are mixed in a complex, non-reversible way during the message schedule expansion before entering the main compression function, thus contributing significantly to the overall cryptographic strength of the SHA-2 family.


In [14]:
def small_sigma1(x: int) -> np.uint32:
    """
    Performs the σ₁ function on a 32-bit word.

    This function is used in the SHA-256 and SHA-224 message schedule expansion.

    The function performs two circular right rotations (ROTR) of the input
    by 17 and 19 positions, one right shift (SHR) by 10 positions, followed 
    by a bitwise XOR of the results.
    
    Parameters:
        x (int): A 32-bit word
        
    Returns:
        np.uint32: The result of ROTR¹⁷(x) ⊕ ROTR¹⁹(x) ⊕ SHR¹⁰(x), as a 32-bit unsigned integer
    """
    # Ensure input is treated as a 32-bit unsigned integer
    x = np.uint32(x)
    
    # Perform circular right rotations by 17 and 19 bits
    rotr17 = np.uint32((x >> 17) | (x << 15)) # 32 - 17 = 15
    rotr19 = np.uint32((x >> 19) | (x << 13)) # 32 - 19 = 13

    # Perform right shift by 10 bits
    # SHR^n(x) = x >> n (fills with zeros on the left)
    shr10 = np.uint32(x >> 10)
    
    # XOR all three values together
    return rotr17 ^ rotr19 ^ shr10

In [15]:
# Test cases to validate the small_sigma1 function
def test_small_sigma1():
  
    test_cases = [
        (0x00000000, 0x00000000), 
        (0xFFFFFFFF, 0x003fffff), 
        (0x80000000, 0x205000), 
        (0x12345678, 0xa1f78649), 
    ]

    for x, expected in test_cases:
        result = small_sigma1(x)
        assert result == expected, (
            f"Test failed for input {hex(x)}: "
            f"expected {hex(expected)}, got {hex(result)}"
        )

    print("All small_sigma1 test cases passed.")

# Run the test
test_small_sigma1()

All small_sigma1 test cases passed.


---

## Problem 2: Fractional Parts of Cube Roots

### Use **NumPy** to calculate the constants listed at the bottom of **page 11 of the Secure Hash Standard**, following the steps below. These are the **first 32 bits of the fractional parts** of the **cube roots of the first 64 prime numbers**.

### Instructions

- Write a function called `primes(n)` that generates the first `n` prime numbers.
- Use the function to calculate the **cube root** of the first 64 primes.
- For each cube root, extract the first thirty-two bits of the fractional part.
- Display the result in **hexadecimal**.
- Test the results against what is in the Secure Hash Standard.

### Solution:

In the Secure Hash Standard, a sequence of 64 constant values is used during the compression function. These constants are derived from the **first 32 bits of the fractional parts of the cube roots** of the first 64 prime numbers [as defined in section 4.2.2 of the standard](https://nvlpubs.nist.gov/nistpubs/fips/nist.fips.180-4.pdf).

This problem walks through how to compute these constants programmatically using NumPy, and verify that they match the official values provided by NIST.

My understanding is that the specific, complex method used to derive these 64 constants is a critical cryptographic design feature. It falls under the principle of using **"nothing up my sleeve"** numbers. So by using the fractional parts of irrational numbers such as the cube roots of primes, the constants are guaranteed to appear **statistically random** meaning that the sequence of bits derived from the fractional part of the cube roots passes tests designed to detect predictable patterns. These would then prevent any suspicion thta the constants were chosen to create a hidden weakness like a backdoor, and ensures that the hash function has strong confusion and diffusion. 

We begin by implementing the `primes(n)` function, which generates the first `n` prime numbers.

For this, I use a basic trial division approach, which is efficient for small values of `n` (such as the first 64 primes). For significantly larger values of `n`, a more advanced algorithm like the [**Sieve of Eratosthenes**](https://www.geeksforgeeks.org/dsa/sieve-of-eratosthenes/) would be more appropriate, however for my implementation it would be slightly over the top.

This implementation handles **edge cases** such as `n ≤ 0`, and includes a **performance optimization:** it skips all even numbers after 2. Additionally, it only checks for divisibility using previously found primes, up to the square root of the candidate number, reducing unnecessary computations.

In [16]:
def primes(n):
    """
    Generates the first n prime numbers using a simple trial division method.
    
    Parameters:
        n (int): The number of prime numbers to generate
        
    Returns:
        list: A list containing the first n prime numbers
    """
    if n <= 0:
        return [] # Return an empty list for non-positive input

    prime_list = [2] # List to store prime numbers starting with the first prime
    candidate = 3 # Start checking for primes from the first odd prime number

    # While loop until we have n primes
    while len(prime_list) < n:
        is_prime = True # Assume candidate is prime until proven otherwise
        for p in prime_list: # Check divisibility with known primes
            if p * p > candidate: 
                break # No need to check further if p^2 exceeds candidate
            if candidate % p == 0:
                is_prime = False # Found a divisor, not prime
                break
        
        if is_prime:
            prime_list.append(candidate) # Add candidate to the list if it's prime
        
        candidate += 2 # Odd numbers only, skip even numbers as we have 2 already

    return prime_list # Return the list of prime numbers

Next, we generate the **first 64 prime numbers** using the `primes(n)` function.

In [17]:
# Generate the first 64 prime numbers
first_64_primes = primes(64)

For each prime number, we calculate the **cube root**, extract its **fractional part**, and multiply it by $2^{32}$ to obtain the first **32 bits** of the fractional part.

These 32-bit values correspond to the **SHA-256 constants** defined in the Secure Hash Standard.

In [18]:
sha256_constants = [] # List to store the K constants

for prime in first_64_primes: 
    cube_root = np.cbrt(prime) # Calculate the cube root
    fractional_part = cube_root - np.floor(cube_root) # Extract the fractional part
    bits32 = np.uint32(fractional_part * (2**32)) # Scale to 32 bits
    sha256_constants.append(bits32) # Append to the constants list

Finally, we display the constants in **hexadecimal format**, grouped in sets of 8, as shown in [section 4.2.2](https://nvlpubs.nist.gov/nistpubs/fips/nist.fips.180-4.pdf) of the **Secure Hash Standard**.

In [19]:
# Display constants in hexadecimal format (8 per line)
for i in range(0, 64, 8):
    print(', '.join(f"0x{k:08x}" for k in sha256_constants[i:i+8])) # Display in hex format

0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5
0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174
0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da
0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967
0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85
0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070
0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3
0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2


The following test verifies that the calculated SHA-256 K constants exactly match the official constants listed in the Secure Hash Standard documentation.  
Passing this test ensures the correctness of the constants used in our SHA-256 implementation.

In [20]:
def test_sha256_constants():
    """
    Test that the SHA-256 K constants match the official constants
    as specified in the SHA-256 standard.
    """
    official_constants = [
        0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
        0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
        0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
        0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
        0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
        0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
        0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
        0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
    ]

    assert sha256_constants == official_constants, "SHA-256 K constants do not match official values"
    print("Test passed: SHA-256 K constants match the official values.")

# Run the test
test_sha256_constants()

Test passed: SHA-256 K constants match the official values.


---

## Problem 3: Padding

### Instructions

- Write a generator function `block_parse(msg)` that processes messages according to section 5.1.1 and 5.2.1 of the Secure Hash Standard. 
- The function should accept a bytes object called `msg`. 
- At each iteration, it should yield the next 512-bit block of `msg` as a bytes object. 
- Ensure that the final block (or final two blocks) include the required padding of `msg` as specified in the standard. 
- Test the generator with messages of different lengths to confirm proper padding and block output.

### Solution:

SHA algorithms operate on fixed-size blocks of data (512 bits for SHA-256). Messages must be padded according to rules defined in **sections 5.1.1 and 5.2.1** of the [Secure Hash Standard](https://nvlpubs.nist.gov/nistpubs/fips/nist.fips.180-4.pdf). As stated in the standard, padding can be inserted before hash computation begins on a message, or at any other time during the hash computation **prior** to processing the block(s) that will contain the padding. This ensures the final message length is compatible with the algorithm’s block size requirements.

The goal of this problem is to implement a generator function, `block_parse(msg)`, that yields successive 512-bit (64-byte) blocks from the input message, correctly applying the necessary padding. 

The padding process for SHA-256 involves three steps to ensure the final padded message length is a multiple of 512 bits.

1. **Append `1` Bit:** The message is terminated by a single `1` bit.
2. **Append `0` Bits (zero padding):** Zero bits are appended until the message length is exactly 448 (mod 512) bits. This step is necessary because the final 64 bits (8    bytes) of the last block are reserved. 
3. **Append Length:** The original message length in bits is appended as a 64-bit big-endian integer. t 

This block-wise processing is critical for the SHA hashing process. Without proper padding, the message would not align with the internal structure of the SHA-256 compression function, potentially leading to incorrect hash outputs.

In [21]:
def block_parse(msg):
    """
    Generator function that parses a message into 512-bit blocks with SHA-256 padding.
    
    Implements padding according to sections 5.1.1 and 5.2.1 of the Secure Hash Standard.
    
    Parameters:
        msg (bytes): The input message to be parsed
        
    Yields:
        bytes: Each 512-bit (64-byte) block of the padded message
    """
    # Length of original message in bits
    msg_len_bits = len(msg) * 8
    
    # Convert message to mutable bytearray for padding 
    padded_msg = bytearray(msg)
    
    # Append the '1' bit (10000000 in binary, or 0x80 in hex)
    padded_msg.append(0x80)
    
    # Calculate 'k' (padding needed) such that the current length (L) + k + 8 is a multiple of 64.
    # This is equivalent to ensuring (L + k) mod 64 = 56.
    # We want a remainder of 56. The modulo 64 handles the wrapping.
    padding_needed = (56 - (len(padded_msg) % 64) + 64) % 64
    
    # Append zero bytes to reach the required length before length encoding
    padded_msg.extend([0x00] * padding_needed)
    
    # Append the original message length as a 64-bit big-endian integer
    padded_msg.extend(msg_len_bits.to_bytes(8, byteorder='big'))
    
    # Yield successive 512-bit (64-byte) blocks
    for i in range(0, len(padded_msg), 64):
        yield bytes(padded_msg[i:i+64])


The following are tests created to ensure the above function works as expected. 

In [22]:
def test_block_parse():
    # Test 1: Empty message
    msg1 = b"" 
    blocks1 = list(block_parse(msg1)) # Convert generator to list for testing
    assert len(blocks1) == 1, "Empty message should produce 1 block"
    expected_block1 = b'\x80' + b'\x00' * 55 + (0).to_bytes(8, 'big') # \x80 is 10000000 in binary, \x00*55 pads to 56 bytes, last 8 bytes are length (0 bits)
    assert blocks1[0] == expected_block1, "Padding for empty message incorrect"

    # Test 2: Short message "abc"
    msg2 = b"abc"
    blocks2 = list(block_parse(msg2))
    assert len(blocks2) == 1, "3-byte message should produce 1 block"
    padding_len = 56 - (len(msg2) + 1)  # +1 for 0x80 byte
    expected_block2 = msg2 + b'\x80' + b'\x00' * padding_len + (len(msg2)*8).to_bytes(8, 'big')
    assert blocks2[0] == expected_block2, "Padding for short message incorrect"

    # Test 3: Boundary case - 55-byte message (one block)
    msg3 = b'a' * 55
    blocks3 = list(block_parse(msg3))
    assert len(blocks3) == 1, "55-byte message should produce 1 block"
    padding_len = 56 - (len(msg3) + 1)
    expected_block3 = msg3 + b'\x80' + b'\x00' * padding_len + (len(msg3)*8).to_bytes(8, 'big')
    assert blocks3[0] == expected_block3, "Padding for 55-byte message incorrect"

    # Test 4: Boundary case - 56-byte message (requires 2 blocks)
    msg4 = b'a' * 56
    blocks4 = list(block_parse(msg4))
    assert len(blocks4) == 2, "56-byte message should produce 2 blocks"

    expected_block4_0 = msg4 + b'\x80' + b'\x00' * 7
    expected_block4_1 = b'\x00' * 56 + (len(msg4)*8).to_bytes(8, 'big')

    assert blocks4[0] == expected_block4_0, "First block for 56-byte message incorrect"
    assert blocks4[1] == expected_block4_1, "Padding block for 56-byte message incorrect"

    # Test 5: Exact 64-byte message (2 blocks)
    msg5 = b'b' * 64
    blocks5 = list(block_parse(msg5))
    assert len(blocks5) == 2, "64-byte message should produce 2 blocks"

    expected_block5_0 = msg5
    expected_block5_1 = b'\x80' + b'\x00' * 55 + (len(msg5)*8).to_bytes(8, 'big')

    assert blocks5[0] == expected_block5_0, "First block for 64-byte message incorrect"
    assert blocks5[1] == expected_block5_1, "Padding block for 64-byte message incorrect"


    print("All block_parse function test cases passed.")

test_block_parse()

All block_parse function test cases passed.


---

## Problem 4: Hashes

### Instructions

- Write a function `hash(current, block)` that calculates the next hash value given the current hash value and the next message block according to section 6.2.2 SHA-256 Hash Computation on page 22 of the Secure Hash Standard.

### Solution:

As explained  [here](https://www.movable-type.co.uk/scripts/sha256.html), a **hash** acts as a kind of digital fingerprint or signature for data. Unlike encryption, hashing is a **one-way** process. It cannot be reversed to recover the original input. No matter how large the input message is, the resulting hash value always has a **fixed size**. 

The `hash_compression(current, block)` function implements the **compression step** at the core of SHA-256. It takes the current 256-bit hash state (eight 32-bit words) and a 512-bit message block, then produces an updated 256-bit state. This allows SHA-256 to process messages of any length by chaining together fixed size blocks.

The function follows [FIPS 180-4 6.2.2](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) and relies on the components defined in the previous problems:

| Component                       | Purpose in Compression                               | From Problems covered above      |
|---------------------------------|------------------------------------------------------|----------------------------------|
| Ch, Maj, Sigma0, Sigma1         | Non-linear mixing functions (mixing and confusion)   | Problem 1                        |
| sigma0, sigma1                  | Message schedule functions (expansion and diffusion) | Problem 1                        |
| SHA constants (sha256_constants)| Round constants derived from cube roots of primes    | Problem 2                        |
| block_parse()                   | Message block preparation(padding and 512-bit blocks)| Problem 3                        |
 
The entire process is broken down into four main steps outlined in the standard: 

1. **Message schedule** – The 512 bit message block is expanded into 64 32 bit words (W0 through W63). The first 16 words come directly from the block. The remaining 48 words are generated using a recurrence relation involving the sigma0 and sigma1 functions (from problem 1) to ensure every bit of the block influences multiple rounds.

2. **Initialize working variables** – The current hash state (H0 through H7) is copied into eight working variables (a through h).

3. **64 rounds of compression** – This would be considered the core of the algorithm, where the working variables are updated 64 times. In each round, new temporary values (T1 and T2) are calculated using the non-linear functions (ch, maj, Sigma0, Sigma1), the round constants K (sha256_constants from problem 2), and the message schedule word Wt. 

4. **Feedforward** – The final values of the working variables (a through h) are added to the initial hash state (H0 through H7) to produce the next 256-bit hash value, whcih becomes the input for the next block's compression. 

Each block’s output becomes the next block’s input, creating a dependency chain that ensures the **avalanche effect**, a single bit change in the message completely alters the final digest. [See here for a high level explanation.](https://www.encryptionconsulting.com/education-center/sha-256/#:~:text=This%20function%20involves%20mixing%20the,value%20and%20the%20message%20block.&text=Repeat%20the%20compression%20function%20for,as%20input%20for%20the%20next.&text=The%20final%20hash%20value%20after,hash%20of%20the%20original%20message.)

The following function follows these exact steps as outlined in the [FIPS 180-4 6.2.2](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf).

**Note:** SHA-256 arithmetic uses 32-bit unsigned integers. When additions exceed the maximum 32-bit value, they wrap around to 0 and NumPy reports this as overflow. NumPy may raise `RuntimeWarning: overflow encountered in scalar add` during these operations. This is expected and does not indicate an error. To suppress these warnings for the following function we can use **np.seterr(over='ignore')**.

In [23]:
# Ignore overflow warnings for SHA-256 arithmetic 
np.seterr(over='ignore'); # See https://numpy.org/devdocs/reference/generated/numpy.seterr.html

In [ ]:
def hash_compression(current: list[np.uint32], block: bytes) -> list[np.uint32]:
    """
    Calculates the next hash value given the current hash and a 512 bit message block.
    
    Implements SHA-256 hash computation according to section 6.2.2 of the Secure Hash Standard.
    
    Note: I have marked each step of the algorithm with comments corresponding to the steps
    in the standard. See section 6.2.2 steps labeled 1-4 https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf

    Parameters:
        current (list): List of 8 np.uint32 values representing the current hash (H0-H7)
        block (bytes): A 512-bit (64-byte) message block
        
    Returns:
        list: List of 8 np.uint32 values representing the updated hash state
    """

    # Ensure current hash values are 32-bit unsigned integers
    H = [np.uint32(h) for h in current]

    # Step 1: Prepare the message schedule (W[0..63])
    W = []

    # First 16 words come directly from the block (big-endian)
    # int.from_bytes converts 4 bytes to a 32-bit integer
    # See: https://docs.python.org/3/library/stdtypes.html#int.from_bytes
    for i in range(16):
        word = int.from_bytes(block[i*4:(i+1)*4], byteorder='big')
        W.append(np.uint32(word))
    
    # Extend the first 16 words into the remaining 48 words
    # Each new word depends on specific earlier words via the small_sigma functions defined in Problem 1
    for i in range(16, 64):
        s0 = small_sigma0(W[i-15])
        s1 = small_sigma1(W[i-2])
        w = np.uint32(W[i-16])
        w = np.uint32(w + s0)
        w = np.uint32(w + W[i-7])
        w = np.uint32(w + s1)
        W.append(w)


    # Step 2: Initialize working variables from current hash values
    a, b, c, d, e, f, g, h = H

    # Step 3: Perform 64 rounds of compression
    for t in range(64):
        # Calculate the two temporary values T1 and T2 with 32-bit wrapping at each addition
        T1 = np.uint32(h)
        T1 = np.uint32(T1 + big_sigma1(e)) # Add big_sigma1(e) (big_sigma1 function defined in Problem 1)
        T1 = np.uint32(T1 + ch(e, f, g)) # Add ch(e,f,g) (ch function defined in Problem 1)
        T1 = np.uint32(T1 + sha256_constants[t]) # Add sha256_constants[t] (sha256_constants defined in Problem 2)
        T1 = np.uint32(T1 + W[t]) # Add W[t]

        # Calculate T2
        T2 = np.uint32(big_sigma0(a)) # Add big_sigma0(a) (big_sigma0 function defined in Problem 1)
        T2 = np.uint32(T2 + maj(a, b, c)) # Add maj(a,b,c) (maj function defined in Problem 1)

        # Update working variables
        h = g
        g = f
        f = e
        e = np.uint32(d + T1)  # Wrap 32-bit addition
        d = c
        c = b
        b = a
        a = np.uint32(T1 + T2)  # Wrap 32-bit addition
    
    # Step 4: Compute the next hash value
    next_hash = [
        np.uint32(H[0] + a),
        np.uint32(H[1] + b),
        np.uint32(H[2] + c),
        np.uint32(H[3] + d),
        np.uint32(H[4] + e),
        np.uint32(H[5] + f),
        np.uint32(H[6] + g),
        np.uint32(H[7] + h)
    ]
    
    # Return updated hash values as a list of 8 np.uint32 values
    return next_hash


In [ ]:
# Test the hash function
def test_hash():
    # SHA-256 initial hash values (first 32 bits of fractional parts of square roots of first 8 primes)
    # Official values can be found in section 5.3.3 of the Secure Hash Standard
    # https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf
    initial_hash = [
        np.uint32(0x6a09e667),
        np.uint32(0xbb67ae85),
        np.uint32(0x3c6ef372),
        np.uint32(0xa54ff53a),
        np.uint32(0x510e527f),
        np.uint32(0x9b05688c),
        np.uint32(0x1f83d9ab),
        np.uint32(0x5be0cd19)
    ]

    # Test message
    msg = b"Nobody inspects the spammish repetition"

    # Split and pad message into 512-bit blocks
    # Using the block_parse function defined in Problem 3
    blocks = list(block_parse(msg))

    # Process each block in sequence
    current_hash = initial_hash[:]
    for block in blocks:
        current_hash = hash_compression(current_hash, block)

    # Produce final digest (concatenate hex values)
    result = ''.join(f'{h:08x}' for h in current_hash)

    # Use hashlib to compute expected SHA-256 hash for verification
    # .sha256() creates a new sha256 hash object
    # .hexdigest() returns the digest as a hexadecimal string
    # See: https://docs.python.org/3/library/hashlib.html
    expected = hashlib.sha256(msg).hexdigest()

    print(f"Result:   {result}")
    print(f"Expected: {expected}")

    assert result == expected, "SHA-256 hash does not match expected value."
    print("SHA-256 hash matches expected value.")

test_hash()


Result:   031edd7d41651593c5fe5c006fa5752b37fddff7bc4e843aa6af0c950f4b9406
Expected: 031edd7d41651593c5fe5c006fa5752b37fddff7bc4e843aa6af0c950f4b9406
SHA-256 hash matches expected value.


---

## Problem 5: Passwords

### Instructions

- The following are the SHA-256 hashes of three common passwords that have been hashed using one pass of the SHA-256 algorithm. 
- As strings, they were encoded using UTF-8. 
- Determine the passwords and explain how you found them. 
- Suggest ways in which the hashing of passwords could be improved to prevent the kind of attack you performed to find the passwords.

### SHA-256 hashes 
1. 5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8
2. 873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34
3. b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342

### Solution:

SHA-256 is a one way cryptographic hash function, meaning it cannot be reversed to obtain the original password. Therefore, the only practical way to discover the passwords is to guess likely candidates, hash them, and compare the results to the target hashes.

To do this, we can perform a [dictionary attack](https://www.techtarget.com/searchsecurity/definition/dictionary-attack) where we use a list of common passwords and check whether their SHA-256 hashes match any of the target hashes.

The list of common passwords I used can be found [here](https://www.kaggle.com/datasets/wjburns/common-password-list-rockyoutxt/data), please note that I am only using the first 10,000 passwords in this file and not the full 14,341,564.

In [26]:
# The three SHA-256 hashes to crack
# Store them as hexadecimal strings in a list
target_hashes = [
    "5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8",
    "873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34",
    "b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342"
]

In [27]:
# Path to the file containing common passwords
password_file = "common_10000_passwords.txt"

This code goes through a list of 10,000 common passwords to see if any of them match the three passwords we are trying to crack.  

Step by step:  
1. Open the file containing the 10,000 passwords.  
2. Read it line by line so we can check each password.  
3. Remove any extra spaces or newline characters from each password.  
4. Hash the password using SHA-256 (built-in `hashlib` function).  
5. Compare the hash to the three target hashes.  
6. If it matches, print the password and its hash.  

Note: Checking 10,000 passwords is fast, but if the list were millions of passwords or if the passwords were very long, this could take much longer.


In [28]:
# Open the file and check each password
with open(password_file, "r", encoding="utf-8") as file:
    for line in file:
        candidate_password = line.strip()  # Remove newline characters and extra spaces

        # Compute the SHA-256 hash of the candidate password
        # See https://docs.python.org/3/library/hashlib.html
        password_hash = hashlib.sha256(candidate_password.encode("utf-8")).hexdigest()

        # Check if the computed hash matches any of the target hashes
        if password_hash in target_hashes:
            print(f"Found password: {candidate_password} -> {password_hash}")

Found password: password -> 5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8
Found password: cheese -> 873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34


Found password: P@ssw0rd -> b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342


**How to improve the hashing of passwords against dictionary attacks:**

**1. Salting:**

According to [OWASP Guidelines](https://cheatsheetseries.owasp.org/cheatsheets/Password_Storage_Cheat_Sheet.html#salting), a salt is a unique, random value generated by a [cryptographically secure function](https://cheatsheetseries.owasp.org/cheatsheets/Cryptographic_Storage_Cheat_Sheet.html) and added to each password before hashing.

**Why salting matters:**
- Ensures identical passwords produce different hashes
- Prevents rainbow table attacks (precomputed hash databases)
- Forces attackers to crack each password individually
- Adds non-determinism to the hashing process

**Example:**
If the password is `farm1990M0O` and a random salt is `f1nd1ngn3m0`:
- Without salt: `hash("farm1990M0O")` -> always the same hash
- With salt: `hash("f1nd1ngn3m0" + "farm1990M0O")` -> unique hash

The salt is stored alongside the hash in the database (it doesn't need to be secret, its randomness provides the security).

**2. Use Proper [Password Hashing Algorithms](https://cheatsheetseries.owasp.org/cheatsheets/Password_Storage_Cheat_Sheet.html#password-hashing-algorithms):**

SHA-256 is designed for speed and data integrity, not password storage. Instead, use:
- [**Argon2**](https://cheatsheetseries.owasp.org/cheatsheets/Password_Storage_Cheat_Sheet.html#argon2id) 
    - Winner of the 2015 Password Hashing Competition.
    - Resists both side-channel attacks and GPU-based cracking.
    - Configurable parameters: memory usage, number of iterations, and parallelism.
    - Example: balances CPU and RAM usage for stronger protection.
- [**scrypt**](https://cheatsheetseries.owasp.org/cheatsheets/Password_Storage_Cheat_Sheet.html#scrypt)
    - Memory-hard key derivation function.
    - Slows down attackers using specialized hardware.
    - Configurable parameters: CPU/memory cost, block size, and parallelism.
- [**bcrypt**](https://cheatsheetseries.owasp.org/cheatsheets/Password_Storage_Cheat_Sheet.html#bcrypt)
    - Suitable for legacy systems if Argon2 or scrypt are not available.
    - Configurable work factor determines how slow the hash is.
    - Maximum password input length is 72 bytes for most implementations.
- [**PBKDF2**](https://cheatsheetseries.owasp.org/cheatsheets/Password_Storage_Cheat_Sheet.html#pbkdf2)
    - NIST-recommended, widely supported, FIPS-140 validated.
    - Uses HMAC internally (SHA-256 is common).
    - Configurable iteration count acts as the work factor.

These algorithms are intentionally slow, making brute force attacks impractical. For new applications, **Argon2id is recommended** as the modern standard.


---
# End of Assessment